In [ ]:
from typing import TypedDict, Annotated, List, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from typing import TypedDict, Annotated, List, Literal, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
import random
from datetime import datetime
from langgraph.graph.message import add_messages # ----> reducer

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

os.environ["GOOGLE_API_KEY"] = google_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key 

llm = init_chat_model("google_genai:gemini-2.0-flash")

E0000 00:00:1760440988.003337   37472 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    
graph_builder = StateGraph(State)

def chatbot(state:State):
    return {"messages": [llm.invoke(state["messages"])]}
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

### Atomic Prompts

In [7]:
def generate(prompt: str) -> str:
    response = graph.invoke({
        "messages": [HumanMessage(content=prompt)]
    })
    return response["messages"][-1].content
response = generate("Write a joke about programmers.")
print(response)

Why do programmers prefer dark mode?

Because light attracts bugs!


### Prompt with a constraint

In [8]:
response = generate("Write a joke about programmers that has to do with them not leaving their chair for a long time.")
print(response)

Why did the programmer's doctor recommend regular stretching?

Because after debugging for 72 hours straight, they found out their legs had become a permanent part of their chair. They were officially "integrated."


#### We can write few shot or whatever too, or assign a role (usually the system prompt things)(im too lazy to do all of that)

#### Let's try a system prompt

In [9]:
system_prompt = """ 
        You are a comedian that likes to tell stories before delivering a punchline. You are always funny. 
        Jokes contain 3 sections:
        1. A setup
        2. A punchline
        3. A contradiction
        Always maintain a jovial tone.
"""

def generate_with_system(prompt: str) -> str:
    response = graph.invoke({
        "messages": [
            SystemMessage(content=system_prompt),
            HumanMessage(content=prompt)
        ]
    })
    return response["messages"][-1].content
response = generate_with_system("Write a joke about programmers that has to do with them not leaving their chair for a long time.")
print(response)

Alright, buckle up, buttercups, because I've got a tale for you that's funnier than a rubber chicken in a library!

So, picture this: There's this programmer, right? A real coding cowboy, name of "Binary" Bob. Now, Bob, he loved coding more than squirrels love nuts. He'd sit in his chair, surrounded by empty energy drink cans and enough snack wrappers to build a small fort, and just *code*. Days turned into nights, nights turned into... well, more nights. His apartment slowly started to resemble a digital cave, and his chair? Oh, that chair became an extension of his very being.

One day, Bob's mom calls, bless her heart. She's worried sick, hasn't seen him in weeks. So, she drives over, knocks on the door, and after a bit of yelling, Bob finally grunts and shuffles to the door. Mom bursts in, takes one look at the scene, and gasps, "Binary Bob! You haven't left this chair in days, have you?!"

Bob, without missing a beat, swivels around, points at the chair, and says:

**"Of course I 

### Sometimes it might not follow this format,,, what then,,,?

In [ ]:
import dspy
lm = dspy.LM("gemini/gemini-2.0-flash", api_key=google_api_key)
dspy.configure(lm=lm)